<a href="https://colab.research.google.com/github/Ali-Alameer/Deep-Learning/blob/main/week7_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train YOLO11 on a Custom Dataset

### **Steps Covered in this Tutorial**

To train our detector we take the following steps:

* Install YOLO11 dependencies
* Load custom dataset from Roboflow in YOLO11 format
* Run YOLO11 training
* Evaluate YOLO11 performance
* Run YOLO11 inference on test images
* OPTIONAL: Deployment
* OPTIONAL: Active Learning


### Preparing a Custom Dataset

If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 1

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
!nvidia-smi

## Preparing a custom dataset

### Step 1: Creating project

Before you start, you need to create a Roboflow [account](https://app.roboflow.com/login). Once you do that, you can create a new project in the Roboflow [dashboard](https://app.roboflow.com/). Keep in mind to choose the right project type. In our case, Object Detection.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/creating-project.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929799852"
  >
</div>

### Step 2: Uploading images

Next, add the data to your newly created project. You can do it via API or through our [web interface](https://docs.roboflow.com/adding-data/object-detection).

If you drag and drop a directory with a dataset in a supported format, the Roboflow dashboard will automatically read the images and annotations together.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/uploading-images.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929808290"
  >
</div>

### Step 3: Labeling

If you only have images, you can label them in [Roboflow Annotate](https://docs.roboflow.com/annotate).

<div align="center">
  <img
    width="640"
    src="https://user-images.githubusercontent.com/26109316/210901980-04861efd-dfc0-4a01-9373-13a36b5e1df4.gif"
  >
</div>

### Step 4: Generate new dataset version

Now that we have our images and annotations added, we can Generate a Dataset Version. When Generating a Version, you may elect to add preprocessing and augmentations. This step is completely optional, however, it can allow you to significantly improve the robustness of your model.

<div align="center">
  <img
    width="640"
    src="https://media.roboflow.com/preparing-custom-dataset-example/generate-new-version.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1673003597834"
  >
</div>

### Step 5: Exporting dataset

Once the dataset version is generated, we have a hosted dataset we can load directly into our notebook for easy training. Click `Export` and select the `desired` dataset format.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/export.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672943313709"
  >
</div>




In [ ]:
!pip install roboflow ultralytics

# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLO11 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.


In [ ]:
!curl -L "https://app.roboflow.com/ds/PEtxKedfAz?key=kVGmcyjqOg" -o roboflow.zip
!unzip roboflow.zip -d roboflow


# Begin Custom Training

We're ready to start custom training.

NOTE: We will only modify one of the YOLO11 training defaults in our example: `epochs`. We will adjust from 30 to 10 epochs in our example for speed.

In [ ]:
# run this cell to begin training; for real implemenation increase number of epochs
!yolo detect train model=yolo11n.pt data=roboflow/data.yaml epochs=5 imgsz=640


In [ ]:
from IPython.display import Image, display

display(Image(filename='/content/runs/detect/train/results.png'))

# Evaluation

We can evaluate the performance of our custom training using the provided evalution script. Make sure the exp #number is up to date -- check the folder within

In [ ]:
# Run evaluation
!yolo predict \
  model=/content/runs/detect/train/weights/best.pt \
  source=/content/roboflow/test/images \
  project=/content/runs/detect \
  name=pred_test \
  exist_ok=True


In [ ]:
import gradio as gr
import glob
from PIL import Image

# folder where YOLO saves predictions
PRED_DIR = "/content/runs/detect/pred_test"

# fetch list of image paths (JPG and PNG support)
images = sorted(glob.glob(f"{PRED_DIR}/*.jpg")) + sorted(glob.glob(f"{PRED_DIR}/*.png"))

def show_image(image_path):
    return Image.open(image_path)

with gr.Blocks() as demo:
    gr.Markdown("## 🐄 YOLO Detection Results Viewer")

    dropdown = gr.Dropdown(
        choices=images,
        label="Select a prediction image",
        value=images[0] if images else None
    )

    image_display = gr.Image(label="Detected Image")

    dropdown.change(fn=show_image, inputs=dropdown, outputs=image_display)

demo.launch(debug=True)
